### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [1]:
!pip install langchain


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
!pip install langchain_groq


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=os.getenv("GROQ_API_KEY"))

In [7]:
!pip install langchain_core


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [9]:
result

AIMessage(content='Here\'s the translation, along with some variations:\n\n* **Formal:**\n    * Bonjour. Comment allez-vous ? (Hello. How are you?)\n\n* **Informal:**\n    * Salut. Comment vas-tu ? (Hello. How are you?) \n\nLet me know if you\'d like to know how to say "Hello" and "How are you?" in different regions of France! 😊\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 89, 'prompt_tokens': 19, 'total_tokens': 108, 'completion_time': 0.161818182, 'prompt_time': 0.00132324, 'queue_time': 0.25192433, 'total_time': 0.163141422}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--40eb803e-0d24-47da-a418-f5065d32aee7-0', usage_metadata={'input_tokens': 19, 'output_tokens': 89, 'total_tokens': 108})

In [10]:
result.content

'Here\'s the translation, along with some variations:\n\n* **Formal:**\n    * Bonjour. Comment allez-vous ? (Hello. How are you?)\n\n* **Informal:**\n    * Salut. Comment vas-tu ? (Hello. How are you?) \n\nLet me know if you\'d like to know how to say "Hello" and "How are you?" in different regions of France! 😊\n'

In [11]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'Here\'s the translation, along with some variations:\n\n* **Formal:**\n    * Bonjour. Comment allez-vous ? (Hello. How are you?)\n\n* **Informal:**\n    * Salut. Comment vas-tu ? (Hello. How are you?) \n\nLet me know if you\'d like to know how to say "Hello" and "How are you?" in different regions of France! 😊\n'

#### Using LCEL - chain from input till output

In [12]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'Here\'s how to say "Hello, how are you?" in French:\n\n* **Formal:**\n\nBonjour, comment allez-vous ? \n\n* **Informal:**\n\nSalut, comment vas-tu ? (used with friends and family)\n\n\nLet me know if you\'d like to know how to say other greetings in French!  \n'

In [13]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)

In [14]:
result = prompt.invoke({"language":"French","text":"Hello How are you?"})
result.to_messages()

[SystemMessage(content='Trnaslate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello How are you?', additional_kwargs={}, response_metadata={})]

In [15]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'